In [2]:
import pandas as pd

df = pd.read_excel('EIDOS_taxonomia.xlsx')[['nameid', 'name', 'taxonrank', 'taxonomicgroup']]

In [46]:
df = df[df['taxonrank'].isin(['Species', 'Subspecies']) & (df['nameid'] < 100000)]
df['chosen'] = False
df['words'] = df['name'].str.split(' ').apply(lambda x: [word.replace('.', '').replace(',', '').replace('(', '').replace(')', '').lower() for word in x if len(word) > 2])
df['all_words'] = df['words'].apply(lambda x: set(x))

In [90]:
df.sample(5, random_state=0)

,nameid,name,taxonrank,taxonomicgroup,chosen,words,all_words,chosen_words
11560,11655.0,Lonchura malacca,Species,Aves,False,"[lonchura, malacca]","{lonc, lonch, l, malacc, lonchur, lonchu, mal,...","{lonc, lonch, l, malacc, lonchur, lonchu, mal,..."
8216,8224.0,Sedum rupestre L.,Species,Plantas vasculares,False,"[sedum, rupestre]","{rupestre, ru, rup, se, r, sedu, s, rupest, ru...","{rupestre, ru, rup, se, r, sedu, s, rupest, ru..."
58800,62432.0,Dicranota flammatra,Species,Invertebrados,False,"[dicranota, flammatra]","{flamma, dicr, fla, dicran, d, flam, flammat, ...","{flamma, dicr, fla, dicran, d, flam, flammat, ..."
81291,85058.0,Oscillatoria formosa,Species,Cromistas y Bacterias,False,"[oscillatoria, formosa]","{oscillat, f, osc, oscillator, oscil, formos, ...","{oscillat, f, osc, oscillator, oscil, formos, ..."
32506,36026.0,"Pseudosquillopsis cerisii (Roux, 1828)",Species,Invertebrados,False,"[pseudosquillopsis, cerisii, roux, 1828]","{pseu, p, pseudosq, 1, pse, ps, c, pseudo, rou...","{pseu, p, pseudosq, 1, pse, ps, c, pseudo, rou..."


In [89]:
all_words = set()
all_words_dict = {}
MIN_LENGTH = 3
for i, row in df.iterrows():
    if not isinstance(row['words'], list):
        continue
    words_to_add = [word for word in row['words'] if len(word) > MIN_LENGTH - 1 and isinstance(word, str)]
    all_words.update(words_to_add)
    to_add = []
    for word in words_to_add:
        to_add += [word[:length+1] for length in range(len(word))]
    all_words.update(to_add)
    all_words_dict[i] = set(to_add)
df['all_words'] = df.index.map(all_words_dict)
df['chosen_words'] = df['all_words']
all_words = sorted(all_words, key=lambda x: len(x), reverse=True) # Las palabras mas largas primero
word_counts = pd.Series(
    [word for words in df['all_words'] if isinstance(words, set) for word in words]
).value_counts()
len(all_words)

273725

In [91]:
try:
    words_to_exclude = set(pd.read_csv('palabras_para_filtrar.csv', header=None)[0])
except:
    words_to_exclude = set()
df['chosen'] = df['all_words'].apply(lambda words: isinstance(words, set) and bool(words & words_to_exclude))
df.loc[df['chosen'], 'chosen_words'] = [set() for _ in range((df['chosen']).sum())]
word_counts_only_chosen = pd.Series(
    [word for words in df['chosen_words'] if isinstance(words, set) for word in words]
).value_counts()
words_df = pd.DataFrame({'all': word_counts, 'chosen': word_counts_only_chosen}).fillna(0)
words_df['chosen'] = words_df['chosen'].astype(int)
words_df[words_df['all'] < 1000].sort_values('chosen', ascending=False).head()

,all,chosen
bra,995,995
pol,982,982
201,957,957
meigen,956,956
meige,956,956
